In [1]:
# TODO: 
#    1. get address link directly without using loggerWebAPI.tcl
#    2. use SDDI format instead of csv

import urllib.request, subprocess
import numpy as np

# this is to get web-page link
# "loggerWebAPI.tcl" can be downloaded from the link below
# https://ops.aps.anl.gov/downloads/loggerWebAPI.tcl

#address = subprocess.getoutput('tclsh loggerWebAPI.tcl -printAPI exportCSV junk.csv 9/29/2022 10/10/2022 6ID ID:6:tc32:TC32:Ti0 ID:6:tc32:TC32:Ti2 ID:6:tc32:TC32:Ti4 ID:6:tc32:TC32:Ti8')
address = subprocess.getoutput('tclsh loggerWebAPI.tcl -printAPI exportCSV junk.csv 10/11/2022 10/11/2022 1ID S:SRcurrentAI')
#address = subprocess.getoutput('tclsh loggerWebAPI.tcl -printAPI exportCSV junk.csv 10/11/2022 10/11/2022 1ID ID01DSEnergy')
#address = subprocess.getoutput('tclsh loggerWebAPI.tcl -printAPI exportCSV junk.csv 10/11/2022 10/11/2022 1ID BL01:SRID:HPosition BL01:SRID:VPosition')

print(address)

# get the csv file from web page
# this part of code is modified from here https://stackoverflow.com/questions/16283799/how-to-read-a-csv-file-from-a-url-with-python/62614979#62614979
response = urllib.request.urlopen(address)
lines = [l.decode('utf-8') for l in response.readlines()]

# header
header = lines[2].rstrip().split(',')

# read data
arr=np.array([])
for i in range(3,len(lines)):
    da_=lines[i].rstrip().split(',')
    if not da_[0]=='':   # skip empty line
        arr = np.append(arr, np.array(da_).astype('float'))

# reshape array, now we have the data
arr = np.reshape(arr,(-1,len(header)))

#print(len(lines), arr.shape, arr.dtype, arr[0,1])

https://ops.aps.anl.gov/cgi-bin/oagMonitorDataReview.cgi?DataGroup=1ID&1ID_ControlReadbackName=18&ExportCSV=Export+Data+(CSV)&StartYear=2022&StartMonth=10&StartDay=11&StartHour=0&EndYear=2022&EndMonth=10&EndDay=11&EndHour=24&Size=mpng&Background=onwhite&LabelSize=0.03&Sparsing=1


In [9]:
#### this part is to plot the result

## convert epoch time to human readable time
#import datetime

eptime = arr[:,1]

## plot result
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib qt5
#%matplotlib inline

#humantime = mdate.date2num(datetime.utcfromtimestamp(eptime)) 

fig,ax=plt.subplots()
for i in range(2, len(header)):
    ax.plot(eptime.astype('datetime64[s]'), arr[:,i],'o', label=header[i])

# set proper timezone to display correct time. 
ax.xaxis_date(tz='US/Central')

plt.xlabel('Time (sec)')
plt.ylabel('Temperature(degC)')
ax.legend()

plt.show()



In [3]:
# get list of datagroup
address = "https://ops.aps.anl.gov/cgi-bin/oagMonitorDataReview.cgi?ListDataGroups=1"

response = urllib.request.urlopen(address)
#lines = [l.decode('utf-8') for l in response.readlines()]
body = response.read()
response.close()
#print(body)